# Charon4 txt to Python
This script loads the data exported with charon4. The following screenshot shows the expected settings while exporting.

2018 data are exported processed different, see `charon4_txt_to_python_2018data.ipynb`

<img src="./pictures/Charon4_export_window_txt.png" alt="drawing" width="1300"/>

The results of this script are stored in `..\Alsdorf\Daten\my_database`. The goal of this database is to contain all data in an easy to read format.

At the moment this script always loads and saves all data, it does not check which data already is in the existing database. I did not plan yet to implement that

After using this script you should run `my_database_script.ipynb`, which gives an overview of the data.

## Data download in Alsdorf
Daten sind direkt runtergeladen, wenn man ankommt. Aber noch nicht auf dem Gerät gelöscht. Dafür habe ich dann data download seperat angeklickt, so werden dann die Daten auf der Speicherkarte gelöscht. Es wird aber anscheinend nichts doppelt in die Database geladen.

In [1]:
version="v1_5"
# Changelog
#v1_0: has a nice overviwe at the end
#v1_1: add temp data from daniel to data_2019; length are int now; added seperate import for 2018
#v1_2: seperated this notebook into different ones; outsourced some user-defined functions
#v1_3: added import for multiple controller files; removed_data_for_month_avaible variable and plot, 
# removed a bug in merging daniel data in 2019
#v1_4: added sort_index before data save (only saved data is sorted); dropped two nan rows in 2021
# removed measurements per day plots from these script and added them to my_databse_script
#v1_5: 

#???????????Charon4 does not export the last two hours, as selected? But exports 2 hours more at the beginning? 
# Time Zone recalculation within the software?
# --> irgendwann nochmal genauer checken

# hiervon werden nicht mehr alle gebraucht
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
#from matplotlib import colors
#from datetime import date, timedelta
from collections import defaultdict
#from collections import Counter
#import matplotlib.patches as patches
#import matplotlib.dates as mdates
#from shutil import copy2 as copy_file
import pickle

from my_func_mvw.functions import get_abspath, read_pickle, write_pickle
from my_func_mvw.functions_import_my_database import merge_data_year,import_my_database_pickle

print("Some Version information of the imported packages")
print(f"pandas: {pd.__version__}")
print(f"pickle: {pickle.format_version}")
!python --version

Some Version information of the imported packages
pandas: 1.3.4
pickle: 4.0
Python 3.8.12


# Input

In [2]:
#path to working directory
#mit \[-]\ im Pfad funktioniert die get_abspath Funktion nicht
# add additional paths to list, if you exported new data from Charon4
# if you have a lot of different paths to controller files, consider exporting the whole databse from Charon4 again
# this would increase the speed of this script (having less paths to controllers)
# but the export from Charon4 takes some time
# selecting the same repository in Charon4 does not work. Doing that, every file is copied at the end of its counterpart.

############Input###################
path_charon4_exp_txt=r"..\Alsdorf\Daten\DTS_unprocessed\Charon4\charon4_export_as_txt"
path_to_controller_Alsdorf=[
    path_charon4_exp_txt,
    path_charon4_exp_txt + "\\07062021", 
    path_charon4_exp_txt + "\\08062021",
    path_charon4_exp_txt + "\\20210702",
    path_charon4_exp_txt + "\\20210719",
    path_charon4_exp_txt + "\\20210727\Alsdorf",
    path_charon4_exp_txt + "\\20210804\Alsdorf",
    path_charon4_exp_txt + "\\20210809\Alsdorf",
    path_charon4_exp_txt + "\\20210917\Alsdorf",
    path_charon4_exp_txt + "\\20211104\Alsdorf"
    ]

path_charon4_exp_txt=r"..\EONERC\Data\unprocessed\charon4_export_as_txt"
path_to_controller_EONERC=[
    path_charon4_exp_txt + "\\EON ERC",
    path_charon4_exp_txt + "\\02112021\EON ERC"
]

data_save_csv=True # True False; data saving takes about 9 min.
data_save_pickle=True #True False; data saving takes about 25 s.
# 3188 is in Alsdorf; 3195 in EONERC
controller=3188 #3188 3195
######################################

# pickle files are much faster saved and read, but need slightly more disc space
# csv seems to be more robust to version changes of packages than pickle
# this gives a nice short overview about different file types: 
# https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

In [3]:
# My old Inputs, dont change them. I am not sure how good they work.
if controller==3188:
    path_to_controller = path_to_controller_Alsdorf
    channels=[1,2,3,4,5,6,7,8] #[1,2,3,4,5,6,7,8]
    years=[2019,2020,2021] #[2019,2020,2021] # 2018 is imported seperate from another data format
    months=[1,2,3,4,5,6,7,8,9,10,11,12] #[1,2,3,4,5,6,7,8,9,10,11,12]

if controller==3195:
    path_to_controller = path_to_controller_EONERC
    ########################################################
    channels=[1,2,3,4,5,6,8,9,10,11]
    years=[2019,2020,2021]
    months=[1,2,3,4,5,6,7,8,9,10,11,12]


# generate all requested paths, some paths may be empty
# see here why defaultdict is used:
# https://stackoverflow.com/questions/44992106/how-do-i-create-a-nested-dictionary-under-a-key-that-is-yet-to-exist
paths_for_activate=defaultdict(dict)
for path_controller in path_to_controller:
    for year in years:
        for channel in channels:
            for month in months:
                paths_for_activate[path_controller][f"temp_ch{channel}_year{year}_month{month}"] = path_controller + f"\Controller\{controller}\{channel}\Temperature Data\{year}\{month}"

#paths_for_activate

#Idee
# für jeden controller path seperate dic erstellen und alles einladen
# dic danach mergen
# darauf achten, dass doppelte Messungen am ende nicht auftauchen

# Load data

In [4]:
def import_temp_to_df(path):
    """"""
    #print(path)
    # error_bad_lines, needed for data of 2018, where the length of the cable changes
    # the data is loast at the moment, by adding columns / changing the data file the data could be imported
    # I think this data wont be important anyway
    df = pd.read_csv(path,skiprows=7,decimal=".",delimiter="\t",index_col=0,on_bad_lines='skip',) #error_bad_lines=False,warn_bad_lines=False
    df = df.drop(df.columns[0:2],axis=1)
    df.index.names = ['Date']
    df.index = pd.to_datetime(df.index, infer_datetime_format=True)
    df.rename(columns=lambda x: int(float(x)), inplace=True)
    df.columns.names=["Length [m]"]
    return df

In [5]:
# Import Data

# maybe its best to store not everything in one variable, not sure if thats True. 

def import_path_for_activate(paths_for_activate):
    """imports all files listed in one path_for_activate dic
    paths to empty repositories are not used
    """

    data_2019=defaultdict(dict)
    data_2020=defaultdict(dict)
    data_2021=defaultdict(dict)

    for name in paths_for_activate.keys(): # loop for every channel of every month
        my_path=paths_for_activate[name] + "\*"
        my_path_to_file=get_abspath(my_path)

        # get info about data from name
        partition1=name.partition("_year")[-1].partition("_month")
        year=partition1[0]
        month=partition1[-1]
        channelnumber=name.partition("_year")[0].partition("temp_ch")[-1]

        # check if the month folder contains more than one file 
        if len(my_path_to_file) > 1:
            if name != "temp_ch2_year2019_month7":
                print("one months contains more than one file, script should be checked! Only encountered this ones")
                print(name)
            # adaption of script
            # Import Data, one option
            one_file=import_temp_to_df(my_path_to_file[0])
            for i in range(1,len(my_path_to_file)):
                additional_files=import_temp_to_df(my_path_to_file[i])
                #display(additional_files)
                #display(one_file)
                one_file=pd.concat([one_file,additional_files],axis=0)
                #display(one_file)
            
            ## Store Data in dic - best would be putting this in a user-defined-function
            # or creating an if with >=1
            
            if year == "2019":
                data_2019[channelnumber][month] = one_file

            if year == "2020":
                data_2020[channelnumber][month] = one_file

            if year == "2021":
                data_2021[channelnumber][month] = one_file
        
        #if len(my_path_to_file) == 0: # wenn pfad zu keinem file führt dann wird leere Liste returned
        
        if len(my_path_to_file) == 1:
            
            # Import Data, second option
            one_file=import_temp_to_df(my_path_to_file[0])

            ## Store Data in dic
            
            if year == "2019":
                data_2019[channelnumber][month] = one_file

            if year == "2020":
                data_2020[channelnumber][month] = one_file

            if year == "2021":
                data_2021[channelnumber][month] = one_file

    return data_2019, data_2020, data_2021


In [6]:
# Import data

n_controller_files=len(path_to_controller)
data_2019_parts=[]
data_2020_parts=[]
data_2021_parts=[]
counter=0
for path in path_to_controller:
    data_2019_part, data_2020_part, data_2021_part = import_path_for_activate(paths_for_activate[path])

    data_2019_parts.append(data_2019_part)
    data_2020_parts.append(data_2020_part)
    data_2021_parts.append(data_2021_part)

    counter+=1
print("done")

done


In [7]:
def merge_dics_data(data_A, data_B):
    """merges two different data dics of the same year
    Vorgehen:
    dic mergen, wenn key nicht vorhanden neuen hinzufügen
    wenn key schon vorhanden, zu dataframe hinzufügen, aber prüfen ob eintrag schon existiert
    """
    data_new=defaultdict(dict)

    for channel in channels:
        c=str(channel)

        # if channel is not avaible in one dic but in the other take the data from the dic
        if c not in data_A.keys() and c in data_B.keys():
            data_new[c]=data_B[c]

        elif c not in data_B.keys() and c in data_A.keys():
            data_new[c]=data_A[c]

        # if channel is in both look at months
        elif c in data_B.keys() and c in data_A.keys():
            for month in months:
                m=str(month)

                # if month is not avaible in one dic but in the other take the data from the dic
                if m not in data_A[c].keys() and m in data_B[c].keys():
                    data_new[c][m]=data_B[c][m]

                elif m not in data_B[c].keys() and m in data_A[c].keys():
                    data_new[c][m]=data_A[c][m]
                
                #if month is in both check if dataframes are equal
                elif m in data_B[c].keys() and m in data_A[c].keys():
                    
                    # if both have same index, theay are equal. So I can take one of them
                    # kann ich eigentlich weglassen und immer concaten und drop duplicate machen
                    # so ist aber schneller
                    if len(data_B[c][m].index) == len(data_A[c][m].index):
                        if data_B[c][m].index == data_A[c][m].index: #if len are not eyqual this gives an error
                            data_new[c][m] = data_A[c][m]
                        
                        else: #copy of other else
                            #concat dataframes, drop duplicetes and sort
                            data_new[c][m] = pd.concat([data_A[c][m],data_B[c][m]],axis=0).drop_duplicates()
                            data_new[c][m].sort_index(axis=0,inplace=True)
                            print(f"concat for channel {c}, month {m} (other else)")

                    # else they dont have the same index, one contains more / additional data
                    else:
                        #concat dataframes, drop duplicetes and sort
                        data_new[c][m] = pd.concat([data_A[c][m],data_B[c][m]],axis=0).drop_duplicates()
                        data_new[c][m].sort_index(axis=0,inplace=True)
                        print(f"concat for channel {c}, month {m}")

    return data_new


In [8]:
#Wall time: 40.9 s
# Merge different dics, which were exported seperate from Charon4
#print("2021")
#data_2021 = merge_dics_data(data_2021_parts[0], data_2021_parts[1])
#print("2020")
#data_2020 = merge_dics_data(data_2020_parts[0], data_2020_parts[1])
#print("2019")
#data_2019 = merge_dics_data(data_2019_parts[0], data_2019_parts[1])

def merge_parts(data_20xx_parts):
    """merges all the different parts"""

    data_20xx = data_20xx_parts[0] #final dataframe
    for i in range(1,len(data_20xx_parts)):
        data_20xx = merge_dics_data(data_20xx, data_20xx_parts[i])

    return data_20xx

print("2021")
data_2021 = merge_parts(data_2021_parts)
print("2020")
data_2020 = merge_parts(data_2020_parts)
print("2019")
data_2019 = merge_parts(data_2019_parts)

2021
concat for channel 1, month 5
concat for channel 2, month 5
concat for channel 3, month 5
concat for channel 4, month 5
concat for channel 1, month 6
concat for channel 2, month 6
concat for channel 3, month 6
concat for channel 4, month 6
concat for channel 5, month 6
concat for channel 6, month 6
concat for channel 7, month 6
concat for channel 8, month 6
concat for channel 1, month 6
concat for channel 2, month 6
concat for channel 3, month 6
concat for channel 4, month 6
concat for channel 5, month 6
concat for channel 6, month 6
concat for channel 7, month 6
concat for channel 8, month 6
concat for channel 1, month 6
concat for channel 1, month 7
concat for channel 2, month 6
concat for channel 2, month 7
concat for channel 3, month 6
concat for channel 3, month 7
concat for channel 4, month 6
concat for channel 4, month 7
concat for channel 5, month 6
concat for channel 5, month 7
concat for channel 6, month 6
concat for channel 6, month 7
concat for channel 7, month 6
conca

In [9]:
# rows with nan, why? Should drop them before saving my_database
# maybe due to stopping an running measurement? or my concat?
# did not investigate further yet
if controller==3188:
    data_2021["1"]["4"] = data_2021["1"]["4"].drop(pd.to_datetime('2021-04-26 08:55:50'),axis=0)
    data_2021["1"]["4"] = data_2021["1"]["4"].drop(pd.to_datetime('2021-04-28 08:28:17'),axis=0)

# Load Daniel data and merge it
Mario gave Geophysica these data. They have been uploaded to moodle. They cover a range, where I did find any data in the Charon4 database. Maybe they have been exported with a different pc?

In [10]:
#%%time
#Wall time: 41.3 s
if controller==3188:
    # Import data from daniel
    def import_data_Daniel(path):
        """import for data from sciebo Eduardschacht repository
        copied from compare script
        """
        df = pd.read_csv(path,delimiter = ',',index_col=0, header=7 )
        df = df.drop(df.columns[[0,1]], axis=1)
        df.index=pd.to_datetime(df.index, dayfirst = True).tz_localize(None)
        df.index=df.index.astype(str) # same column format as other dataframe
        df=df.transpose() # for having the same format as the other dataframe
        df.index=df.index.astype(float) #make depth to int
        df.index=df.index.astype(int) #make depth to int
        df.index.names = ['Depth [m]']
        df.columns.names=["Length [m]"] # was wollte ich hier machen?
        return df

    def put_files_in_dataframe(list_of_paths,import_function):
        """
        merges files in one dataframe using an import_function (user-defined-functiom) in a list_of paths (list)
        copied from compare script --> not updated in comare script!!!
        """
        # Import Data
        dic={}
        for path in list_of_paths:
            dic[path]=import_function(path)

        # Merge files into one dataframe
        dic_concat=pd.concat(dic.values(),axis=1)
        dic_concat.columns = pd.to_datetime(dic_concat.columns)
        dic_concat = dic_concat.sort_index(axis=1)

        return dic_concat

    # Import files
    daniel_ch1_paths=get_abspath(r"..\Alsdorf\Daten\DTS_unprocessed\Betrieb2019\Ch1\*")
    daniel_ch2_paths=get_abspath(r"..\Alsdorf\Daten\DTS_unprocessed\Betrieb2019\Ch2\*")
    daniel_ch3_paths=get_abspath(r"..\Alsdorf\Daten\DTS_unprocessed\Betrieb2019\Ch3\*")
    daniel_ch4_paths=get_abspath(r"..\Alsdorf\Daten\DTS_unprocessed\Betrieb2019\Ch4\*")
    # Merge them in one dataframe
    daniel_ch1=put_files_in_dataframe(daniel_ch1_paths,import_data_Daniel)
    daniel_ch2=put_files_in_dataframe(daniel_ch2_paths,import_data_Daniel)
    daniel_ch3=put_files_in_dataframe(daniel_ch3_paths,import_data_Daniel)
    daniel_ch4=put_files_in_dataframe(daniel_ch4_paths,import_data_Daniel)

In [11]:
# Add daniel data to data_2019
if controller==3188:
    def add_daniel_data_to_2019(daniel_chx,channel_of_daniel):
        """
        beide Datensätze sind gleich in dem Bereich wo sie sich überschneiden, siehe compare script

        daniel data will overwrite data in data_2019    
        """
        # Change the format of daniel data, so that it fits to data_2019
        daniel_chx.index.names=[""]
        daniel_chx=daniel_chx.transpose() #----geändert vorher ch2 - war bug!
        daniel_chx.index.names=["Date"]
        #daniel_chx.index=pd.to_datetime(daniel_chx.index) #now in put_files_in_dataframe()

        # Group data by month and create seperate dataframes
        # groupby your key / level and freq
        g = daniel_chx.groupby(pd.Grouper(level='Date', freq='M'))
        # groups to a list of dataframes with list comprehension
        dfs = [group for _,group in g]

        # Add the seperate dataframes from group in a dictionary, so they have the same format as data_2019
        daniel_dic=defaultdict(dict)
        count_month=3 # first month of daniel data, which will used as an overwrite
        for dataframe in dfs[1:]: #skip first one, so count_month fits--------------geändert - war bug!
            daniel_dic[channel_of_daniel][str(count_month)]=dataframe
            count_month+=1
            
        # Add data to 2019 dataframe
        for month in daniel_dic[channel_of_daniel].keys():
            data_2019[channel_of_daniel][month]=daniel_dic[channel_of_daniel][month]
        return daniel_dic
    #Add the data
    print("1")
    daniel_dic_1=add_daniel_data_to_2019(daniel_ch1,"1")
    print("2")
    daniel_dic_2=add_daniel_data_to_2019(daniel_ch2,"2")
    print("3")
    daniel_dic_3=add_daniel_data_to_2019(daniel_ch3,"3")
    print("4")
    daniel_dic_4=add_daniel_data_to_2019(daniel_ch4,"4")

1
2
3
4


# Save Data

In [12]:
#%%time
# Wall time: 12min 25s only csv
# Wall time: 100 s only pickle
def data_save_helper(data_20xx,channel):
    """merges different months to one file"""
    # join all month, so that the data is saved
    first_month=list(data_20xx[channel].keys())[0]
    one_file=data_20xx[channel][first_month]
    for month in list(data_20xx[channel].keys())[1:]: #skip first month, defined above
        one_file = pd.concat([one_file, data_20xx[channel][month]], axis=0)
    # sort all dates, in 2019 is month 9 before 10. Did not yet check how this happened, 
    # but this is solved with this line anyway
    one_file = one_file.sort_index()
    #one_file = round(one_file,2) # not needed
    return one_file

def save_data_csv(data_20xx,save_to_path):
    """"""
    for channel in data_20xx.keys():
        one_file = data_save_helper(data_20xx,channel)
        
        # save data
        year=one_file.index[10].strftime("%Y") #randomly chose 10; very bad solution
        filename=f"\\temp_ch{channel}_{year}.csv"
        one_file.columns.names=["Length [m]"]
        one_file.to_csv(save_to_path + filename)

def write_pickle_custom(save_to:str,data_20xx):
    """I adaptded this function only for this notebook"""
    #Function to write pickle Files
    for channel in data_20xx.keys():
        one_file = data_save_helper(data_20xx,channel)

        # save data
        year=one_file.index[10].strftime("%Y") #randomly chose 10; very bad solution
        filename=f"\\temp_ch{channel}_{year}"
        with open(save_to + filename, 'wb') as handle:
            pickle.dump(one_file, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Which dataformat is best? Thats nice short overview:
# https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d
# Save data
path_to_my_database_Alsdorf = r"..\Alsdorf\Daten\DTS_processed"

path_to_my_database_EONERC = r"..\EONERC\Data\unprocessed\data_from_charon4"

if controller==3188:
    path_to_my_database = path_to_my_database_Alsdorf

if controller==3195:
    path_to_my_database = path_to_my_database_EONERC

# a bít unecessary to save these, if I save charon4 and 3 later in one file for 3195
save_to = path_to_my_database + "\csv"
if data_save_csv:
    save_data_csv(data_2021,save_to)
    save_data_csv(data_2020,save_to)
    save_data_csv(data_2019,save_to)
if data_save_pickle:
    save_to_pickle = path_to_my_database + "\pickle"
    write_pickle_custom(save_to_pickle, data_2021)
    write_pickle_custom(save_to_pickle, data_2020)
    write_pickle_custom(save_to_pickle, data_2019)

    # save all in one file
    # nicht gerade optimal gelöst mit dem neu einladen
    def save_all(controller=controller): #variables will not be saved outside the function
        data_2021=import_my_database_pickle(2021, path_to_my_database + "\pickle",controller=controller)
        data_2020=import_my_database_pickle(2020, path_to_my_database + "\pickle",controller=controller)
        data_2019=import_my_database_pickle(2019, path_to_my_database + "\pickle",controller=controller)
        if controller==3188:
            write_pickle(path_to_my_database + "\import_data_scripts\data_all", merge_data_year([data_2019, data_2020, data_2021]))
        elif controller==3195:
            write_pickle(path_to_my_database + "\..\..\data_all_charon4",merge_data_year([data_2019, data_2020, data_2021]))
    save_all()

In [13]:
#Load data - example
# temp_ch1_2021.csv temp_ch2_2021.csv temp_ch3_2021.csv temp_ch4_2021.csv
# temp_ch1_2020.csv temp_ch2_2020.csv temp_ch3_2020.csv temp_ch4_2020.csv
# temp_ch1_2019.csv temp_ch2_2019.csv temp_ch3_2019.csv temp_ch4_2019.csv
#filename=f"\\temp_ch2_2020.csv" 
#test=pd.read_csv(save_to + filename, index_col=0)
#test

In [14]:
last_month=list(data_2021["1"].keys())[-1]
last_measurement_2021 = data_2021["1"][last_month].index[-1]
last_measurement_2021 #last measurement of channel 1

Timestamp('2021-11-04 10:13:18')